In [ ]:
import os
import pandas as pd
import json
import ast

current_dir = os.getcwd()
root_dir = os.path.dirname(os.path.dirname(current_dir))
_RAWFIRESPATH = f"{root_dir}/RawData/Historical_FiresRAW"

In [ ]:
_year = 2005
_file = f"{_RAWFIRESPATH}/{_year}/2005-01-01_37.7388357_-8.750062888101874.csv"
df = pd.read_csv(f"{_file}")
# print(df.head())
column_names = df.columns.tolist()
print(column_names)

# Get all column names
column_names = df.columns.tolist()

# Find columns that contain 'hourly_units'
columns_to_remove = [col for col in column_names if 'hourly_units' in col]
columns_to_remove.extend(['latitude', 'longitude', 'generationtime_ms', 'utc_offset_seconds', 'timezone', 'timezone_abbreviation'])

# Remove the columns
df = df.drop(columns=columns_to_remove)

# Print the updated column names
updated_column_names = df.columns.tolist()
print(updated_column_names)

_file_radiation = _file.replace(".csv", "_radiation.csv")
df = pd.read_csv(f"{_file_radiation}")
print(df.head())

df = pd.read_csv(f"{_file_radiation}")
# print(df.head())
column_names = df.columns.tolist()
print(column_names)

# Get all column names
column_names = df.columns.tolist()

# Find columns that contain 'hourly_units'
columns_to_remove = [col for col in column_names if 'hourly_units' in col]
columns_to_remove.extend(['latitude', 'longitude', 'generationtime_ms', 'utc_offset_seconds', 'timezone', 'timezone_abbreviation', 'elevation', 'hourly.time', 'hourly.global_tilted_irradiance_instant', 'hourly.diffuse_radiation_instant', 'hourly.terrestrial_radiation_instant', 'hourly.direct_radiation_instant', 'hourly.shortwave_radiation_instant', 'hourly.direct_normal_irradiance_instant'])

# Remove the columns
df = df.drop(columns=columns_to_remove)

# Print the updated column names
updated_column_names_radiation = df.columns.tolist()
print(updated_column_names_radiation)

# Convert the lists to sets
set1 = set(updated_column_names_radiation)
set2 = set(updated_column_names)

# Find the common elements
common_elements = set1 & set2

# Print the common elements
print(common_elements)

In [ ]:
header = ['year', 'date', 'district', 'municipality', 'parish','latitude', 'longitude', 'cause']

columns = header + updated_column_names + updated_column_names_radiation


df = pd.DataFrame(columns=columns)

df.to_csv('b2012.csv', index=False)

In [ ]:
dfsourcex = pd.read_csv(f'{_RAWFIRESPATH}/filtered_icnf_2022.csv')
filtered_df = dfsourcex[(dfsourcex['TIPO'] == 'Florestal') & (dfsourcex['TIPOCAUSA'].isin(['Desconhecida', 'Natural']))]
print(len(filtered_df))

In [ ]:
_year = 2012

df = pd.read_csv("b2012.csv")

print(df.head())

dfsourcex = pd.read_csv(f'{_RAWFIRESPATH}/filtered_icnf_2022.csv')
filtered_df = dfsourcex[(dfsourcex['tipo'] == 'Florestal') & (dfsourcex['tipo_causa'].isin(['Desconhecida', 'Natural']))]
print(len(filtered_df))

second_row = filtered_df.iloc[0]
DIA = second_row['DIA']
MES = second_row['MES']
HORA = second_row['HORA']
LAT = second_row['LAT']
LON = second_row['LON']
ANO = second_row['ANO']
DISTRICTO = second_row['DISTRITO']
CONCELHO = second_row['CONCELHO']
FREGUESIA = second_row['FREGUESIA']

MeteorologyFile = False
RadiationFile = False

print(DIA, MES, HORA, LAT, LON, ANO)

MES = "0" + str(MES) if MES < 10 else str(MES)
DIA = "0" + str(DIA) if DIA < 10 else str(DIA)

_fileName = _DATA + f"_{LAT}_{LON}.csv"

_fileName_radiation = _DATA + f"_{LAT}_{LON}_radiation.csv"

#_fileName_radiation = f"{DIA}_{MES}_{HORA}_{LAT}_{LON}_radiation.csv"

print(_fileName, _fileName_radiation)

isExistMeteorology = os.path.isfile(f'{_RAWFIRESPATH}/{_year}/{_fileName}')
if(isExistMeteorology):
    MeteorologyFile = True

isExistRadiation = os.path.isfile(f'{_RAWFIRESPATH}/{_year}/{_fileName_radiation}')
if(isExistRadiation):
    RadiationFile = True

print(isExistMeteorology, RadiationFile)

if(MeteorologyFile and RadiationFile):
    dfMet = pd.read_csv(f"{_RAWFIRESPATH}/{_year}/{_fileName}")
    dfRad = pd.read_csv(f"{_RAWFIRESPATH}/{_year}/{_fileName_radiation}")

    print(dfMet['elevation'].iloc[0])

    list_from_string = ast.literal_eval(dfMet[updated_column_names[2]].iloc[0])[HORA]

    print(list_from_string)

    list_from_string = ast.literal_eval(dfMet[updated_column_names[1]].iloc[0])[HORA]

    print(list_from_string)

    new_data = {
        'year': ANO,
        'date': f"{ANO}-{MES}-{DIA}",
        'district': DISTRICTO,
        'municipality': CONCELHO,
        'parish': FREGUESIA,
        'latitude': LAT,
        'longitude': LON,
        'elevation': dfMet['elevation'].iloc[0]
    }

    try:
        for i in range(1, len(updated_column_names)):
            new_data[updated_column_names[i]] = ast.literal_eval(dfMet[updated_column_names[i]].iloc[0])[HORA]
    except Exception as e:
        print("->", e)

    try:
        for i in range(0, len(updated_column_names_radiation)):
            new_data[updated_column_names_radiation[i]] = ast.literal_eval(dfRad[updated_column_names_radiation[i]].iloc[0])[HORA]
    except Exception as e:
        print("-x", e)

    new_df = pd.DataFrame(new_data, index=[0])

    new_df.to_csv('b2012.csv', mode='a', header=False, index=False)
